# multi-hazard prediction

loading the NOAA dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
data= pd.read_csv("data/processed_USA_data.csv")

In [ ]:
data

loading pressure level data

In [ ]:
pressure_level_12= pd.read_csv("data/extracted csv_pressure_level/processed_pressure_level_12h.csv")
pressure_level_6= pd.read_csv("data/extracted csv_pressure_level/processed_pressure_level_6h.csv")
pressure_level_4= pd.read_csv("data/extracted csv_pressure_level/processed_pressure_level_4h.csv")
pressure_level_2= pd.read_csv("data/extracted csv_pressure_level/processed_pressure_level_2h.csv")
pressure_level_1= pd.read_csv("data/extracted csv_pressure_level/processed_pressure_level_1h.csv")

In [ ]:
def rename_columns(df, suffix):
    return df.rename(columns={col: f"{col}_{suffix}" for col in df.columns if col != "file_id"})

# Rename columns with unique suffixes
pressure_level_12 = rename_columns(pressure_level_12, "12h")
pressure_level_6 = rename_columns(pressure_level_6, "6h")
pressure_level_4 = rename_columns(pressure_level_4, "4h")
pressure_level_2 = rename_columns(pressure_level_2, "2h")
pressure_level_1 = rename_columns(pressure_level_1, "1h")

# Merge datasets column-wise based on 'file_id'
merged_data_pressure_level = pressure_level_12.merge(pressure_level_6, on="file_id", how="inner") \
                               .merge(pressure_level_4, on="file_id", how="inner") \
                               .merge(pressure_level_2, on="file_id", how="inner") \
                               .merge(pressure_level_1, on="file_id", how="inner")


In [ ]:
# merged_data_pressure_level=pressure_level_1

In [ ]:
pressure_level_12

loading precipation data

In [ ]:
avg_precip=pd.read_csv("data/extracted_preciption_data_csv/era_5_precipitation_avg_data.csv")

In [ ]:
avg_precip

In [ ]:
# Extracting the specified columns
avg_Precip_current = avg_precip[['id', 'avg_precip_current']]
avg_precip_12 = avg_precip[['id', 'avg_precip_12h_early']]
avg_precip_24 = avg_precip[['id', 'avg_precip_24h_early']]


In [ ]:
avg_Precip_current

In [ ]:
data.columns

In [ ]:
NASA_features = data[['event_id', 'landslide_category', 'landslide_trigger','landslide_size','longitude','latitude']]


In [ ]:
NASA_features

In [ ]:

merged_data_pressure_level = merged_data_pressure_level.rename(columns={"file_id": "id"})


In [ ]:
NASA_features = NASA_features.rename(columns={'event_id': 'id'})

Merging data pressure level and NASA Data

In [ ]:

# Merge the DataFrames on the 'ID' column
merged_data = NASA_features.merge(avg_precip, on='id', how='inner').merge(merged_data_pressure_level, on='id', how='inner')

# Display the first few rows of the merged DataFrame to verify



In [ ]:
# Check for NULL or NaN values in the merged_data DataFrame
null_summary = merged_data.isnull().sum()

# Display columns with missing values
print("Columns with NULL or NaN values:")
print(null_summary[null_summary > 0])

# Alternatively, display the total count of missing values
total_missing = merged_data.isnull().sum().sum()
print(f"\nTotal number of missing values: {total_missing}")


In [ ]:
# Remove rows with any NaN values
merged_data = merged_data.dropna()

# Verify that there are no more NaN values
print("Number of missing values after dropping NaN rows:")
print(merged_data.isnull().sum().sum())


In [ ]:
merged_data.head()

In [ ]:
# Calculate and print max, min, and mean of the 'avg_precip_current' column
max_precip = merged_data['avg_precip_current'].max()
min_precip = merged_data['avg_precip_current'].min()
mean_precip = merged_data['avg_precip_current'].mean()

print(f"Maximum value of avg_precip_current: {max_precip}")
print(f"Minimum value of avg_precip_current: {min_precip}")
print(f"Mean value of avg_precip_current: {mean_precip}")


In [ ]:
# Replace values less than 0 with 0 in specified columns
columns_to_update = ['avg_precip_24h_early', 'avg_precip_12h_early', 'avg_precip_current']
merged_data[columns_to_update] = merged_data[columns_to_update].applymap(lambda x: max(x, 0))

# Print confirmation message
print("All negative values in the specified columns have been set to 0.")


In [ ]:
# Count and print the number of values less than 0 in the 'avg_precip_current' column
negative_values_count = (merged_data['avg_precip_current'] < 0).sum()
print(f"Number of values in 'avg_precip_current' less than 0: {negative_values_count}")


In [ ]:
merged_data

Removing all record other than landslide and midslide

In [ ]:
# Display unique values in the 'landslide_category' column
unique_landslide_categories = merged_data['landslide_category'].unique()
print(unique_landslide_categories)


In [ ]:
# Remove rows where 'landslide_category' is 'other'
merged_data = merged_data[merged_data['landslide_category'] != 'other']

# To verify the removal, you can check the unique categories again
print(merged_data['landslide_category'].unique())


In [ ]:
merged_data

LSTM code

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Preparation: Assuming 'merged_data' is already defined and preprocessed

# Encode categorical variables
label_encoders = {}
for column in ['landslide_category', 'landslide_trigger', 'landslide_size']:
    le = LabelEncoder()
    merged_data[column] = le.fit_transform(merged_data[column])

# Define features and targets
feature_columns = [col for col in merged_data.columns if col not in ['id', 'landslide_category', 'landslide_trigger', 'landslide_size', 'avg_precip_current', 'longitude', 'latitude']]
X = merged_data[feature_columns].values
y_loc = merged_data[['latitude', 'longitude']].values

# Scaling features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_loc_scaled = scaler.fit_transform(y_loc)  # Scale location data

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_loc_scaled, test_size=0.2, random_state=42)


In [ ]:
X_train.shape

In [ ]:

# Convert to PyTorch tensors and reshape for LSTM
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).view(-1, 1, X_train.shape[1]).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).view(-1, 1, X_test.shape[1]).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

# Create TensorDataset and DataLoader for training and testing
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
X_train_tensor.shape

predicting location

In [ ]:
# Assuming you have loaded and scaled your data into X_tensor and y_loc_tensor
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_tensor, y_train_tensor, test_size=0.1, random_state=42)

# Ensure the data has a sequence length dimension for LSTM
# Add a sequence length of 1 for LSTM input
# X_train = X_train.unsqueeze(1)  # Add a sequence dimension
# X_val = X_val.unsqueeze(1)      # Add a sequence dimension

# Convert tensors to the correct device
X_train = X_train.to(device)
y_train = y_train.to(device)
X_val = X_val.to(device)
y_val = y_val.to(device)

# Create TensorDataset and DataLoader for the training and validation sets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


class AdvancedLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.5):
        super(AdvancedLSTMModel, self).__init__()
        # Initializing the LSTM with dropout and making it bidirectional
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, 
                            dropout=dropout, bidirectional=True)
        # Adjusting the input feature size for the fully connected layer due to bidirectionality
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # hidden_dim * 2 because it's bidirectional

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(num_layers * 2, x.size(0), hidden_dim).to(device)  # *2 for bidirectionality
        # Initialize cell state
        c0 = torch.zeros(num_layers * 2, x.size(0), hidden_dim).to(device)
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


# Initialize the model, criterion, and optimizer
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = LSTMModel(input_dim=X_train.shape[2], hidden_dim=64, output_dim=2, num_layers=2).to(device)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_dim = X_train.shape[2]  # example feature size
hidden_dim = 64
output_dim = 2  # example output dimension
num_layers = 2
dropout = 0.5

# Model instantiation
model = AdvancedLSTMModel(input_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(device)

In [ ]:
X_train.shape

In [ ]:

# Training and validation loop
num_epochs = 200
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation phase
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_val_loss += loss.item()
    
    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

# Plotting the learning curves
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error

# Validation loop
model.eval()
with torch.no_grad():
    total_loss = 0
    predictions = []
    actuals = []
    
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Forward pass
        outputs = model(X_batch)
        
        # Reverse scaling for accurate performance measurement
        outputs_np = outputs.cpu().numpy()  # Convert to numpy array on CPU
        y_batch_np = y_batch.cpu().numpy()  # Convert to numpy array on CPU
        
        # Apply inverse transformation to scale back to original data space
        outputs_rescaled = scaler.inverse_transform(outputs_np)
        y_batch_rescaled = scaler.inverse_transform(y_batch_np)
        
        # Calculate loss on the original scale
        loss = criterion(torch.tensor(outputs_rescaled), torch.tensor(y_batch_rescaled))
        total_loss += loss.item()
        
        predictions.append(outputs_rescaled)
        actuals.append(y_batch_rescaled)

    # Calculate average loss
    avg_loss = total_loss / len(test_loader)
    print(f'Validation Loss: {avg_loss:.4f}')

    # Flatten the lists for MAE and R2 calculations
    predictions_flat = np.vstack(predictions)
    actuals_flat = np.vstack(actuals)

    # Compute MAE and R2
    mae = mean_absolute_error(actuals_flat, predictions_flat)
    r2 = r2_score(actuals_flat, predictions_flat)
    mse = mean_squared_error(actuals_flat, predictions_flat)
    print(f'Mean Absolute Error: {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')
    print(f'MSE Score: {mse:.4f}')

    # Display first 10 actual and predicted values
    print("First 10 Actual and Predicted Values:")
    for i in range(10):
        print(f"Actual: {actuals_flat[i]}, Predicted: {predictions_flat[i]}")


Predicting the precipation value

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Define features and target
feature_columns = [col for col in merged_data.columns if col not in ['id', 'landslide_category', 'landslide_trigger', 'landslide_size', 'avg_precip_current', 'longitude', 'latitude']]
X = merged_data[feature_columns].values
y_precip = merged_data['avg_precip_current'].values.reshape(-1, 1)  # Reshape for scaling

# Scaling features and target
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_precip_scaled = scaler_y.fit_transform(y_precip)  # Scale precipitation data

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_precip_scaled, test_size=0.2, random_state=42)

# Now X_train, X_test, y_train, y_test are ready to be used for model training and testing


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Assuming 'merged_data' is pre-loaded and contains the column 'avg_precip_current'

# Define features and the target for prediction
feature_columns = [col for col in merged_data.columns if col not in ['id', 'landslide_category', 'landslide_trigger', 'landslide_size', 'avg_precip_current', 'longitude', 'latitude']]
X = merged_data[feature_columns].values
y_precip = merged_data['avg_precip_current'].values.reshape(-1, 1)  # Ensure it's a 2D array for scaling

# Scaling features and target
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_precip_scaled = scaler_y.fit_transform(y_precip)

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_precip_scaled, test_size=0.1, random_state=42)

# Convert arrays to tensors and ensure correct shape for LSTM
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1).to(device)
X_val = torch.tensor(X_val, dtype=torch.float32).unsqueeze(1).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_val = torch.tensor(y_val, dtype=torch.float32).to(device)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [ ]:

# Define the LSTM model adapted for single value output (precipitation prediction)
class AdvancedLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.5):
        super(AdvancedLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Adjust for bidirectionality

    def forward(self, x):
        h0 = torch.zeros(num_layers * 2, x.size(0), hidden_dim).to(device)
        c0 = torch.zeros(num_layers * 2, x.size(0), hidden_dim).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Model instantiation for precipitation prediction
input_dim = X_train.shape[2]
hidden_dim = 64
output_dim = 1  # Predicting a single value
num_layers = 2
dropout = 0.5

model = AdvancedLSTMModel(input_dim, hidden_dim, output_dim, num_layers, dropout).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training and validation loop
num_epochs = 200
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_val_loss += loss.item()
    
    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')

# Plotting the learning curves


In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Assume 'scaler_y' is the MinMaxScaler instance used to scale the 'avg_precip_current' during preprocessing

# Validation loop
model.eval()
with torch.no_grad():
    total_loss = 0
    predictions = []
    actuals = []
    
    for X_batch, y_batch in val_loader:  # Ensure you use the correct DataLoader here, i.e., val_loader
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Forward pass
        outputs = model(X_batch)
        
        # Convert tensors to numpy arrays for inverse scaling
        outputs_np = outputs.cpu().numpy()  # Convert to numpy array on CPU
        y_batch_np = y_batch.cpu().numpy()  # Convert to numpy array on CPU
        
        # Apply inverse transformation to scale back to original data space
        outputs_rescaled = scaler_y.inverse_transform(outputs_np)
        y_batch_rescaled = scaler_y.inverse_transform(y_batch_np)
        
        # Collect predictions and actual values for metric calculations
        predictions.append(outputs_rescaled)
        actuals.append(y_batch_rescaled)

    # Flatten the lists for metrics calculation
    predictions_flat = np.vstack(predictions)
    actuals_flat = np.vstack(actuals)

    # Compute metrics
    mae = mean_absolute_error(actuals_flat, predictions_flat)
    r2 = r2_score(actuals_flat, predictions_flat)
    mse = mean_squared_error(actuals_flat, predictions_flat)

    # Print performance metrics
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'R2 Score: {r2:.4f}')
    print(f'Mean Squared Error (MSE): {mse:.4f}')

    # Display first 10 actual and predicted values
    print("First 10 Actual and Predicted Values:")
    for i in range(10):
        print(f"Actual: {actuals_flat[i][0]:.4f}, Predicted: {predictions_flat[i][0]:.4f}")


predicting landslide category and trigger point

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Device configuration for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming 'merged_data' is already defined and preprocessed

# Encode categorical variables
label_encoders = {}
categorical_columns = ['landslide_category', 'landslide_trigger', 'landslide_size']
for column in categorical_columns:
    if column in merged_data.columns:
        le = LabelEncoder()
        merged_data[column] = le.fit_transform(merged_data[column])
        label_encoders[column] = le  # Store label encoder for potential inverse transformation

# Define features and new targets
feature_columns = [col for col in merged_data.columns if col not in ['id', 'landslide_category', 'landslide_trigger', 'landslide_size', 'avg_precip_current', 'longitude', 'latitude']]
X = merged_data[feature_columns].values
y_category = merged_data['landslide_category'].values
y_trigger = merged_data['landslide_trigger'].values

# Scaling features
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

# Split data into train and test sets for both targets
X_train, X_test, y_cat_train, y_cat_test = train_test_split(X_scaled, y_category, test_size=0.2, random_state=42)
_, _, y_trig_train, y_trig_test = train_test_split(X_scaled, y_trigger, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_cat_train = torch.tensor(y_cat_train, dtype=torch.long)
y_cat_test = torch.tensor(y_cat_test, dtype=torch.long)
y_trig_train = torch.tensor(y_trig_train, dtype=torch.long)
y_trig_test = torch.tensor(y_trig_test, dtype=torch.long)

# Create TensorDataset and DataLoader for training
train_dataset_cat = TensorDataset(X_train, y_cat_train)
train_loader_cat = DataLoader(train_dataset_cat, batch_size=64, shuffle=True)
train_dataset_trig = TensorDataset(X_train, y_trig_train)
train_loader_trig = DataLoader(train_dataset_trig, batch_size=64, shuffle=True)


In [ ]:

# Define an example neural network model for classification
class ClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ClassificationModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.softmax(self.layer2(x))
        return x

# Number of unique categories and triggers
num_categories = len(np.unique(y_category))
num_triggers = len(np.unique(y_trigger))

# Initialize the model for landslide category prediction
model_cat = ClassificationModel(input_dim=len(feature_columns), hidden_dim=100, output_dim=num_categories).to(device)
model_trig = ClassificationModel(input_dim=len(feature_columns), hidden_dim=100, output_dim=num_triggers).to(device)

# Setup the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_cat = torch.optim.Adam(model_cat.parameters(), lr=0.001)
optimizer_trig = torch.optim.Adam(model_trig.parameters(), lr=0.001)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setup data loaders for testing
test_dataset_cat = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_cat_test, dtype=torch.long))
test_loader_cat = DataLoader(test_dataset_cat, batch_size=64, shuffle=False)

test_dataset_trig = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_trig_test, dtype=torch.long))
test_loader_trig = DataLoader(test_dataset_trig, batch_size=64, shuffle=False)

# Training function
def train_model(model, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Testing function
def test_model(model, test_loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
    return y_true, y_pred

# Run training and testing
num_epochs = 200
for epoch in range(num_epochs):
    loss_cat = train_model(model_cat, train_loader_cat, optimizer_cat, criterion)
    loss_trig = train_model(model_trig, train_loader_trig, optimizer_trig, criterion)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss Cat: {loss_cat:.4f}, Loss Trig: {loss_trig:.4f}')

# Testing and classification report for category
y_true_cat, y_pred_cat = test_model(model_cat, test_loader_cat)
print("Classification Report for Landslide Category:")
print(classification_report(y_true_cat, y_pred_cat))

# Testing and classification report for trigger
y_true_trig, y_pred_trig = test_model(model_trig, test_loader_trig)
print("Classification Report for Landslide Trigger:")
print(classification_report(y_true_trig, y_pred_trig))
